## ATIVIDADE 1 - Web Scraping em Ambiente Controlado

Importações de **bibliotecas** utilizadas

In [1]:
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import time
import csv

Definindo as **variáveis globais** que serão utilizadas ao longo do código

In [2]:
baseUrl = 'http://127.0.0.1:8000/places/default'

ids = ['places_country__row', 'places_currency_name__row', 'places_continent__row', 'places_neighbours__row']
pages = set()
info = []

## Funções

Função para pegar os links de páginas dos **países** (a partir da página *index*)

In [3]:
def getLinks(pageUrl):
    global pages, baseUrl
    fullUrl = urljoin(baseUrl, pageUrl)

    try:
        html = urlopen(fullUrl)
        bs = BeautifulSoup(html, 'html.parser')

        for link in bs.find_all('a'):
            href = link.get('href')
            newPage = urljoin(baseUrl, href)
            if newPage not in pages and 'user' not in href:
                if 'index' in href or 'view' in href:
                    pages.add(newPage)
                    getLinks(href)
    except HTTPError as e:
        print(f'Erro HTTP [{e.code}] tentando acessar [{fullUrl}]')
    except Exception as e:
        print(f'Deu pau :( [{e}]')

Função **pega_nome** trabalha juntamente com a **procura_info**, onde a última pega todas as informações necessárias das páginas - *Nome, Continente, Moeda* - e a primeira, além de entrar nos links dos países vizinho, pega o nome deles

In [4]:
def pega_nome(classe):
    aux = []
    for vizinho in classe.find_all('a'):
        url = urljoin(baseUrl, vizinho['href'])
        html = urlopen(url)
        bs = BeautifulSoup(html, 'html.parser')
        
        if 'iso' in url:
            linha = bs.find(id='places_country__row')
            if linha:
                nome = linha.find(class_='w2p_fw')
                if nome:
                    aux.append(nome.text.strip())
            
    return aux        

In [5]:
def procura_info(aux, bs):
    global ids, info
    
    for i in ids:
        linha = bs.find(id=i)
        classe = linha.find(class_='w2p_fw')

        if i == 'places_neighbours__row':
            aux.append(pega_nome(classe))
        else:
            aux.append(classe.text.strip())

    aux.append(time.ctime())
    info.append(aux)

Salva as informações presentes no array em um **.csv**

In [22]:
def salva_csv():
    global info
    colunas = ['País', 'Moeda', 'Continente', 'Países Vizinhos', 'Horário']

    with open('Paises.csv', 'w', newline='') as arq:
        w = csv.writer(arq)
        w.writerow(colunas)
        w.writerows(info)

print('Deu certo')

Deu certo


## Main

Partes de código em que as funções são chamadas e testes são realizados

In [7]:
getLinks('/places/default/index')

In [8]:
for page in pages:
    if 'index' not in page:
        aux = []
        html = urlopen(page)
        bs = BeautifulSoup(html, 'html.parser')

        procura_info(aux, bs)
    

In [9]:
for i in info:
    print(i)

['Monaco', 'Euro', 'EU', ['France'], 'Thu Aug 22 14:02:20 2024']
['Afghanistan', 'Afghani', 'AS', ['Turkmenistan', 'China', 'Iran', 'Tajikistan', 'Pakistan', 'Uzbekistan'], 'Thu Aug 22 14:02:21 2024']
['American Samoa', 'Dollar', 'OC', [], 'Thu Aug 22 14:02:22 2024']
['Angola', 'Kwanza', 'AF', ['Democratic Republic of the Congo', 'Namibia', 'Zambia', 'Republic of the Congo'], 'Thu Aug 22 14:02:23 2024']
['Saint Kitts and Nevis', 'Dollar', 'NA', [], 'Thu Aug 22 14:02:23 2024']
['Sao Tome and Principe', 'Dobra', 'AF', [], 'Thu Aug 22 14:02:23 2024']
['Honduras', 'Lempira', 'NA', ['Guatemala', 'Nicaragua', 'El Salvador'], 'Thu Aug 22 14:02:24 2024']
['Faroe Islands', 'Krone', 'EU', [], 'Thu Aug 22 14:02:24 2024']
['Morocco', 'Dirham', 'AF', ['Algeria', 'Western Sahara', 'Spain'], 'Thu Aug 22 14:02:25 2024']
['Uruguay', 'Peso', 'SA', ['Brazil', 'Argentina'], 'Thu Aug 22 14:02:25 2024']
['Suriname', 'Dollar', 'SA', ['Guyana', 'Brazil', 'French Guiana'], 'Thu Aug 22 14:02:26 2024']
['Chile',

In [23]:
salva_csv()